In [5]:
import pandas as pd
import pickle
import os
import osmnx as ox
import folium
import pydeck as pdk
import networkx as nx
from tqdm import tqdm
import copy
import random
import numpy as np
import itertools

In [ ]:

# Глобальная переменная для веса ребра (время в пути, которое подготовил Человек 1)
WEIGHT_KEY = 'travel_time'

class PercolationSimulator:
    """
    Класс для симуляции перколяционного анализа транспортной сети.
    """
    def __init__(self, G, od_pairs):
        """
        Инициализация симулятора.
        G: Неориентированный граф (должен быть создан из MultiDiGraph, как в osmnx).
        od_pairs: Список критических пар (Origin-Destination) для расчета Criticality.
        """
        self.original_graph = G
        self.initial_nodes_count = len(G.nodes)
        self.initial_edges_count = len(G.edges)
        self.od_pairs = od_pairs

        print(f"Симулятор инициализирован. Узлов: {self.initial_nodes_count}, Ребер: {self.initial_edges_count}")
        # Шаг 1: Расчет базовых кратчайших путей (d_ij^original)
        self.baseline_distances = self._calculate_baseline_distances()

    # --- МЕТРИКИ УСТОЙЧИВОСТИ (Y-ОСЬ) ---

    def calculate_lcc_fraction(self, graph):
        """
        Считает долю узлов в крупнейшей связной компоненте (LCC).
        """
        if len(graph.nodes) == 0:
            return 0

        # Удаляем изолированные узлы, если они остались после удаления ребер
        G_temp = graph.copy()
        G_temp.remove_nodes_from(list(nx.isolates(G_temp)))

        if len(G_temp.nodes) == 0:
            return 0

        components = list(nx.connected_components(G_temp))
        largest_component = max(components, key=len, default=set())

        return len(largest_component) / self.initial_nodes_count

    def calculate_global_efficiency(self, graph):
        """
        Считает Глобальную Эффективность (E).
        E = (1 / N(N-1)) * sum(1 / d_ij)
        """
        if len(graph.nodes) <= 1:
            return 0.0

        # Вычисляем кратчайшие пути, используя вес 'travel_time'
        # Если пути нет, расстояние = Infinity
        sum_inverse_dist = 0.0

        # Используем только те пары, которые были в OD (для ускорения и соответствия Criticality Score)
        for u, v in self.od_pairs:
            try:
                # Находим кратчайший путь по времени (travel_time)
                d_uv = nx.shortest_path_length(graph, source=u, target=v, weight=WEIGHT_KEY)
                # Добавляем инверсию расстояния (1/d)
                sum_inverse_dist += 1.0 / d_uv
            except nx.NetworkXNoPath:
                # Если путь разорван, вклад в сумму = 0
                sum_inverse_dist += 0.0
            except nx.NodeNotFound:
                # Если узел был удален, пропускаем
                continue

        # Нормализуем на количество пар (N * (N-1))
        # Используем количество пар, а не N*(N-1) для точности, так как берем подмножество OD
        if not self.od_pairs:
             return 0.0

        return sum_inverse_dist / len(self.od_pairs)

    # МЕТРИКА ВАЖНОСТИ (ОСНОВА ЦЕЛЕВОЙ АТАКИ)

    def _calculate_baseline_distances(self):
        """
        Предварительный расчет кратчайших путей для всех OD пар (d_ij^original).
        """
        print("Предрасчет исходных расстояний...")
        distances = {}
        for u, v in tqdm(self.od_pairs, desc="Базовый D_ij"):
            try:
                # Расчет кратчайшего пути по travel_time
                d_uv = nx.shortest_path_length(self.original_graph, source=u, target=v, weight=WEIGHT_KEY)
                distances[(u, v)] = d_uv
            except nx.NetworkXNoPath:
                distances[(u, v)] = np.inf
            except nx.NodeNotFound:
                # Это не должно произойти в исходном графе, но на всякий случай
                distances[(u, v)] = np.inf
        return distances

    def calculate_criticality_scores(self):
        """
        Расчет Критичности Ребра (We) по формуле Stretch Factor.
        Чем выше We, тем важнее ребро.
        """
        print("Расчет Criticality Scores (Stretch Factor)... Это займет время.")

        edges_to_score = list(self.original_graph.edges(data=False))
        scores = {}

        for u_edge, v_edge in tqdm(edges_to_score, desc="Criticality Score"):
            G_temp = self.original_graph.copy()
            # Удаляем ребро. В неориентированном графе нет ключа (u, v)
            G_temp.remove_edge(u_edge, v_edge)

            total_stretch_factor = 0.0

            for u, v in self.od_pairs:
                if u == v: continue # Пропускаем

                # 1 Считаем d_ij^new (путь после удаления ребра)
                try:
                    d_new = nx.shortest_path_length(G_temp, source=u, target=v, weight=WEIGHT_KEY)
                except nx.NetworkXNoPath:
                    # Если путь разорван, d_new = бесконечность
                    d_new = np.inf
                except nx.NodeNotFound:
                    continue

                # 2 Получаем d_ij^original из предрасчета
                d_original = self.baseline_distances.get((u, v), np.inf)

                # 3 Расчет Stretch Factor (удлинения)
                if d_original != np.inf and d_original > 0:
                    # Удлинение пути (d_new - d_original) относительно исходного пути
                    stretch = (d_new - d_original) / d_original
                    # Берем max(0, ...) — учитываем только, если путь удлинился
                    total_stretch_factor += max(0, stretch) * 1.0 # *1.0 - здесь можно добавить Спрос_ij
                elif d_original == np.inf and d_new != np.inf:
                    # Если путь изначально был разорван, но после удаления стал не разорванным
                    # (очень редкий случай), просто пропускаем
                    pass
                elif d_original != np.inf and d_new == np.inf:
                     # Если ребро разорвало единственный путь, это бесконечный Stretch.
                     # Добавляем большой штраф (например, 100)
                     total_stretch_factor += 100.0

            scores[(u_edge, v_edge)] = total_stretch_factor

        # Сортируем ребра: от самых важных (высокий скор) к неважным
        sorted_edges = sorted(scores.items(), key=lambda item: item[1], reverse=True)
        return [edge[0] for edge in sorted_edges]

    # ЦИКЛ СИМУЛЯЦИИ

    def simulate_percolation(self, strategy='random', steps=20):
        """
        Основной цикл симуляции.
        strategy: 'random' или 'targeted'.
        """
        G_temp = self.original_graph.copy()

        # 1. Получаем порядок удаления ребер
        if strategy == 'random':
            # Для Random: просто перемешиваем все ребра
            edges_to_remove = list(G_temp.edges(data=False))
            random.shuffle(edges_to_remove)
            # Для Random нужно усреднение, но для простоты здесь один прогон
        else:
            # Для Targeted: считаем Criticality Score (W_e)
            edges_to_remove = self.calculate_criticality_scores()

        total_edges = len(edges_to_remove)
        results = []
        chunk_size = max(1, total_edges // steps)

        print(f"\nЗапуск симуляции: {strategy}. Удаляем по {chunk_size} ребер.")

        # Начальная точка (0% удалено)
        results.append({
            'fraction_removed': 0.0,
            'lcc_fraction': self.calculate_lcc_fraction(G_temp),
            'global_efficiency': self.calculate_global_efficiency(G_temp),
            'strategy': strategy
        })

        removed_count = 0

        for i in tqdm(range(0, total_edges, chunk_size)):
            chunk = edges_to_remove[i : i + chunk_size]

            # Удаляем пачку ребер
            G_temp.remove_edges_from(chunk)
            removed_count += len(chunk)

            # Рассчитываем и записываем метрики
            fraction_removed = removed_count / total_edges

            results.append({
                'fraction_removed': fraction_removed,
                'lcc_fraction': self.calculate_lcc_fraction(G_temp),
                'global_efficiency': self.calculate_global_efficiency(G_temp),
                'strategy': strategy
            })

            # Оптимизация: если связность или эффективность упала, можно закончить
            if results[-1]['global_efficiency'] < 0.05:
                break

        return pd.DataFrame(results)

# ИМИТАЦИЯ ДАННЫХ И ЗАПУСК

def _create_toy_graph():
    """
    Создает маленький игрушечный граф, имитирующий городскую сеть.
    Узел 1: Центр (критичен)
    Ребро (2,3) и (4,1): "Мост" и "Магистраль" (должны получить высокий W_e)
    """
    G = nx.Graph()

    # Добавляем узлы
    nodes = [1, 2, 3, 4, 5, 6, 7, 8]
    G.add_nodes_from(nodes)

    # Добавляем ребра с весом (travel_time). Меньший вес = быстрее.
    # Магистрали
    G.add_edge(2, 3, travel_time=2)   # Критический мост/узел
    G.add_edge(4, 1, travel_time=3)   # Критический путь в центр

    # Дороги с нормальным трафиком
    G.add_edge(1, 2, travel_time=5)
    G.add_edge(3, 4, travel_time=5)

    # Объездные/спальные дороги (дольше)
    G.add_edge(5, 6, travel_time=8)
    G.add_edge(6, 7, travel_time=8)
    G.add_edge(7, 8, travel_time=8)

    # Соединение
    G.add_edge(2, 5, travel_time=6)
    G.add_edge(4, 7, travel_time=7)

    # Создаем OD пары: поездки из спальных районов (5, 8) в Центр (1)
    od_pairs = [(5, 1), (8, 1), (5, 7), (6, 4)]

    print("Игрушечный граф создан.")
    return G, od_pairs

if __name__ == "__main__":
    # 1 ЗАГРУЗКА ИЛИ СОЗДАНИЕ ГРАФА
    try:
        # Попытка загрузить реальный граф
        G_raw = nx.read_graphml("./map_data/moscow_weighted.graphml")
        G = nx.Graph(G_raw) # Преобразование в неориентированный Graph (для LCC)
        # Здесь нужно загрузить реальные od_pairs
        # od_pairs = pd.read_csv("./data/od_pairs.csv").to_records(index=False).tolist()

        # Если загрузка удалась, используем все пары узлов
        od_pairs = list(itertools.product(G.nodes, G.nodes))
        # Ограничиваем для больших графов:
        od_pairs = [pair for pair in od_pairs if random.random() < 0.01] # 1% от всех пар

        if not od_pairs:
             print("Нет OD пар, генерируем случайные 1000 пар.")
             nodes = list(G.nodes)
             od_pairs = [(random.choice(nodes), random.choice(nodes)) for _ in range(1000)]


    except Exception:
        print("Файл графа от Человека 1 не найден. Используем ИГРУШЕЧНЫЙ ГРАФ.")
        G, od_pairs = _create_toy_graph()

    # 2 ИНИЦИАЛИЗАЦИЯ И СИМУЛЯЦИЯ

    sim = PercolationSimulator(G, od_pairs)

    # 2.1. Сценарий 1: Целевая атака (удаляем самые "критичные" дороги)
    results_targeted = sim.simulate_percolation(strategy='targeted', steps=20)

    # 2.2. Сценарий 2: Случайный отказ (удаляем случайные дороги)
    results_random = sim.simulate_percolation(strategy='random', steps=20)

    # 3 ФИНАЛЬНЫЙ ВЫВОД

    final_df = pd.concat([results_targeted, results_random])
    output_path = "results/percolation_results.csv"

    import os
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    final_df.to_csv(output_path, index=False)

    print(f"Сохранено в: {output_path}")

Файл графа от Человека 1 не найден. Используем ИГРУШЕЧНЫЙ ГРАФ.
Игрушечный граф создан.
Симулятор инициализирован. Узлов: 8, Ребер: 9
Предрасчет исходных расстояний...


Базовый D_ij: 100%|██████████| 4/4 [00:00<00:00, 33756.97it/s]


Расчет Criticality Scores (Stretch Factor)... Это займет время.


Criticality Score: 100%|██████████| 9/9 [00:00<00:00, 9999.67it/s]



Запуск симуляции: targeted. Удаляем по 1 ребер.


 11%|█         | 1/9 [00:00<00:00, 4156.89it/s]



Запуск симуляции: random. Удаляем по 1 ребер.


 33%|███▎      | 3/9 [00:00<00:00, 5980.47it/s]


✅ Готово! Результаты для Человека 3 сохранены в: results/percolation_results.csv
